In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
torch.__version__

'1.0.1'

In [2]:
BATCH_SIZE = 512
EPOCHS = 20

In [3]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [4]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out

In [16]:
model = ConvNet()
optimizer = optim.Adam(model.parameters())
print(model)

ConvNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=2000, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [15]:
params = list(model.parameters())
print(params)

[Parameter containing:
tensor([[[[-1.5692e-01,  9.1475e-02,  5.2455e-02,  1.7846e-01,  2.1577e-01],
          [ 2.3178e-01,  3.1371e-01,  8.5955e-02, -1.8129e-01, -1.1215e-01],
          [ 1.5970e-02, -1.1045e-01, -9.1522e-02,  6.2954e-02, -7.4004e-02],
          [-1.1620e-01, -1.1466e-01, -2.2851e-01,  1.7146e-01,  2.7885e-02],
          [-5.6950e-02, -4.8372e-02,  1.0013e-02,  5.5083e-05, -1.3445e-01]]],


        [[[ 9.2209e-02, -1.8548e-01,  8.7760e-02, -8.0922e-02,  3.9589e-02],
          [ 1.3693e-01, -1.6525e-01, -3.0846e-01,  1.7289e-02,  1.3112e-01],
          [ 2.3782e-01, -1.5258e-01, -1.2569e-01, -2.1420e-01, -1.3906e-01],
          [-1.3411e-01,  2.4165e-01,  1.4294e-01,  6.6360e-02, -1.4799e-01],
          [ 7.1282e-02,  2.4951e-01,  3.7682e-02,  1.9280e-01,  9.0307e-02]]],


        [[[-7.8362e-02,  1.6348e-01,  7.5093e-02,  1.2273e-02,  1.3231e-01],
          [-1.4507e-01,  2.2347e-01,  1.5418e-01,  1.4340e-01,  2.1347e-01],
          [ 2.3604e-01,  2.1383e-01, -2.2437e

In [11]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [12]:
def test(model,test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [13]:
for epoch in range(1, EPOCHS + 1):
    train(model,train_loader, optimizer, epoch)
    test(model,test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.315953
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.256822
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.109840

Test set: Average loss: 0.0964, Accuracy: 9698/10000 (97%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.103537
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.080122
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.080824

Test set: Average loss: 0.0573, Accuracy: 9810/10000 (98%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.093928
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.051579
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.041413

Test set: Average loss: 0.0395, Accuracy: 9867/10000 (99%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.052614
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.031835
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.048364

Test set: Average loss: 0.0463, Accuracy: 9855/10000 (99%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.031126
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.028782
Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.040739

Test 